# FLOTO application development

This notebook will launch a container on CHI@Edge, on a Raspberry Pi, in a environment very similar to FLOTO. 

A limitation of this environment is that, CHI@Edge does not set up volumes between containers, meaning you won't be able to set up a container to collect data and a separate container to upload data, as may be done on FLOTO. 
However, on CHI@Edge, you will be able to interact with your container and view its logs, which will allow you to verify the service is working as expected.

## Setup 

First, you'll need to enter your Chameleon project ID in the input after running the next cell.

In [ ]:
import chi
chi.use_site("CHI@Edge")
chi.set("project_name", input("Please enter your project ID (CHI-XXXXXX)").strip())

## Start container

Next, we'll start your application in a container, just as would be done on FLOTO. We set the FLOTO environment variables, which are automatically set when a job is run. If your service requires other environment variables, add them as entries in the `environment` dict. 

In [ ]:
from uuid import uuid4

environment = {
    "FLOTO_JOB_UUID": str(uuid4()),
    "FLOTO_DEVICE_UUID": str(uuid4()),
}

Now, set the image reference for the container in the variable below. This is the place where the container image is pulled from (i.e. what you would `docker pull`). 

The given value here is set to a demo "data collector", which just writes randomly generated data to the shared FLOTO volume (which in this CHI@Edge environment, isn't a shared volume). You can browse the source in `demo_container/`

In [ ]:
container_ref = "mppowers/collector"

Finally, we actually start the container on CHI@Edge. First, we'll need to make a reservation for a Raspberry Pi, and then we launch our container using it.

In [ ]:
from chi import lease
from chi.container import create_container, destroy_container

reservations = []
lease.add_device_reservation(reservations, count=1, machine_name="raspberrypi4-64")
edge_lease = lease.create_lease(
    "floto_lease", 
    reservations, 
    start_date='now', 
    # Note, by default this runs for 2 hours! If you need more time, increase the value here.
    end_date=lease.lease_duration(days=0,hours=2)[1]
)
print("Waiting for lease to start")
lease.wait_for_active(edge_lease["id"])
print("Ready to continue!")

In [ ]:
from chi.container import destroy_container
try:
    print("Launching container. This may take some time for the image to download on the device")
    floto_container = create_container(
        name="floto-container",
        image=container_ref,
        environment=environment,
        reservation_id=lease.get_device_reservation(edge_lease["id"]),
    )
except RuntimeError as ex:
    print(ex)
    try:
        destroy_container(floto_container.uuid)
        print("Automatically cleaned up container, please try again.")
    except:
        print(f"Please stop and/or delete floto_container and try again")
else:
    print(f"Successfully created container: floto_container!")

# Interacting with the container

Now, we can execute commands within the container, and view its logs. The demo container I mentioned earlier expects the `/share` directory to exist, since in FLOTO it'll be created automatically. We can create that manually in this test environment.

In [ ]:
from chi.container import execute

print(execute(floto_container.uuid, "mkdir -p /share && touch /share/log.txt")["output"])

You can view your container's metadata on the CHI@Edge dashboard: https://chi.edge.chameleoncloud.org/project/container/containers

If you select the container, you can also access the "logs" tab, which will show the logs from the container, or the "console" tab which will give you an interactive console into the container, allowing you to execute commands inside a shell.

You can also get the logs in this python environment. Here, we get the last 10 lines.

In [ ]:
from chi.container import get_logs

print("\n".join(x for x in get_logs(floto_container.uuid).splitlines()[-10:]))